In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_regression, make_swiss_roll
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR

In [51]:
df = pd.read_csv("tedsd_puf_2019.csv")
df.head()

,DISYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,LOS,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2019,20191553576,2,-9,4,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1
1,2019,20191465214,2,-9,3,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
2,2019,20191443889,2,-9,2,1,7,-9,35,1,...,0,0,0,0,0,0,9,4,0,3
3,2019,20191409377,2,-9,3,1,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,3
4,2019,20191479567,2,-9,3,3,7,-9,37,1,...,0,0,0,0,0,0,9,4,0,1


In [53]:
df1 = df.copy()
# from conversation with "house of Hope recovery" class 4 which is "transfer to a different facility" would also be considered a success.
for value in df1['REASON']:
    if value == 1:
        value = value
    elif value == 4:
        value = 1
    else:
        value = 0
df1['REASON'].notna().value_counts()

REASON
True    1722503
Name: count, dtype: int64

In [54]:
# time to clean up the data starting with SUB1_D 
# alcohol could be a success, mmj could be a success

for value in df1['SUB1_D']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
df1['SUB1_D'].notna().value_counts()

SUB1_D
True    1722503
Name: count, dtype: int64

In [90]:
# time to clean up the data starting with SUB1 
# alcohol could be a success, mmj could be a success, using the same values for SUB1_D

for value in df1['SUB1']:
    if value == 1 or value == 14 or value == 16 or value == 18 or value == 12 or value == 13 or value == 2:
        value = 1
    else:
        value = 0
df1['SUB1_D'].notna().value_counts()

SUB1_D
True    1722503
Name: count, dtype: int64

In [55]:
# time to clean up the data starting with ARRESTS and ARRESTS_D 
# alcohol could be a success, mmj could be a success

for value in df1['ARRESTS']:
    if value == -9:
        value = 0
    else:
        value = value
for value in df1['ARRESTS_D']:
    if value == -9:
        value = 0
    else:
        value = value
df1['ARRESTS'].notna().value_counts()

ARRESTS
True    1722503
Name: count, dtype: int64

In [56]:
# time to clean up the data with FREQ1_D, FREQ2_D, FREQ3_D
# replacing -9 in the columns with 1 since 1 indicates 'no use' and if data was missing we assumed it was irrelivent to begin with
df1['FREQ1_D'] = df1['FREQ1_D'].replace(-9, 1)
df1['FREQ2_D'] = df1['FREQ2_D'].replace(-9, 1)
df1['FREQ3_D'] = df1['FREQ3_D'].replace(-9, 1)

In [57]:
# time to clean up the data with FREQ1, FREQ2, FREQ3
# replacing -9 in the columns with 1 since 1 indicates 'no use' and if data was missing we assumed it was irrelivent to begin with
df1['FREQ1'] = df1['FREQ1'].replace(-9, 1)
df1['FREQ2'] = df1['FREQ2'].replace(-9, 1)
df1['FREQ3'] = df1['FREQ3'].replace(-9, 1)

In [58]:
# time to clean up the data with 'CASEID'
df1['CASEID'].notna().value_counts()
# CASEID has no null values and does need info filled

CASEID
True    1722503
Name: count, dtype: int64

In [60]:
# time to clean up the data with 'AGE'
df1['AGE'].notna().value_counts()
# AGE has no null values and does need info filled

AGE
True    1722503
Name: count, dtype: int64

In [61]:
# time to clean up the data with 'GENDER'
# gender is not a columns where we can fill in the data and not decrease the accuracy. so for all -9 values I am replacing with 0
df1['GENDER'] = df1['GENDER'].replace(-9, 0)
df1['GENDER'].notna().value_counts()

GENDER
True    1722503
Name: count, dtype: int64

In [ ]:
# time to clean up the data with 'DISYR'
df1['DISYR'].notna().value_counts()
# DISYR has no null values and does need info filled

In [62]:
# time to clean up the data with 'RACE'
# for race, '7' indicateds 'other single race  so for all -9 values i replaces them with 7
df1['RACE'] = df1['RACE'].replace(-9, 7)
df1['RACE'].notna().value_counts()

RACE
True    1722503
Name: count, dtype: int64

In [64]:
# time to clean up the data with 'ETHNIC'
# for ETHNIC, '4' indicateds 'other single race  so for all -9 values I replaced them with 4
df1['ETHNIC'] = df1['ETHNIC'].replace(-9, 4)
df1['ETHNIC'].notna().value_counts()

ETHNIC
True    1722503
Name: count, dtype: int64

In [65]:
# time to clean up the data with 'MARSTAT'
# for MARSTAT, hard to determine what to fill -9 with so filling with 0 for now
df1['MARSTAT'] = df1['MARSTAT'].replace(-9, 0)
df1['MARSTAT'].notna().value_counts()

MARSTAT
True    1722503
Name: count, dtype: int64

In [66]:
# time to clean up the data with 'EDUC'
# for MARSTAT, hard to determine what to fill -9 with. options are 1-5 so filling with 2 as it seems like a fair average
df1['EDUC'] = df1['EDUC'].replace(-9, 2)
df1['EDUC'].notna().value_counts()

EDUC
True    1722503
Name: count, dtype: int64

In [75]:
# time to clean up the data with 'EMPLOY' and 'EMPLOY_D'
# for EMPLOY, hard to determine what to fill -9 with. options are 1-4. filling with 2 for now since if the data is missing is seems like that would indicate 'unemployed'
df1['EMPLOY'] = df1['EMPLOY'].replace(-9, 2)
df1['EMPLOY_D'] = df1['EMPLOY_D'].replace(-9, 2)
df1['EMPLOY'].notna().value_counts()

EMPLOY
True    1722503
Name: count, dtype: int64

In [74]:
# time to clean up the data with 'DETNLF' and 'DETNLF_D'
# for DETNLF, its just details on the previous columns not in labor force option so im replacing missing values with 0 as to not affect the results as much. we may want to drop this column
df1['DETNLF'] = df1['DETNLF'].replace(-9, 0)
df1['DETNLF_D'] = df1['DETNLF_D'].replace(-9, 0)
df1['DETNLF'].notna().value_counts()

DETNLF
True    1722503
Name: count, dtype: int64

In [71]:
# time to clean up the data with 'PREG'
# for PREG, -9 most likely indicates male patients. replacing with 0 as to not affect the results as much. 
df1['PREG'] = df1['PREG'].replace(-9, 0)
df1['PREG'].notna().value_counts()

PREG
True    1722503
Name: count, dtype: int64

In [72]:
# time to clean up the data with 'VET'
# for VET, -9 most likely indicates not a veteren so im replacing -9 with 2 for 'no'
df1['VET'] = df1['VET'].replace(-9, 2)
df1['VET'].notna().value_counts()

VET
True    1722503
Name: count, dtype: int64

In [76]:
# time to clean up the data with 'LIVARAG'
# for LIVARAG, its hard to determine what -9 could indicate so im replacing -9 with 0 as to not affect the data as much with the -9's in this column
df1['LIVARAG'] = df1['LIVARAG'].replace(-9, 0)
df1['LIVARAG_D'] = df1['LIVARAG_D'].replace(-9, 0)
df1['LIVARAG'].notna().value_counts()

LIVARAG
True    1722503
Name: count, dtype: int64

In [77]:
# time to clean up the data with 'PRIMINC'
# for PRIMINC, its hard to determine what -9 could indicate so im replacing -9 with 0 as to not affect the data as much with the -9's in this column
df1['PRIMINC'] = df1['PRIMINC'].replace(-9, 0)
df1['PRIMINC'].notna().value_counts()

PRIMINC
True    1722503
Name: count, dtype: int64

In [78]:
# time to clean up the data with 'ARRESTS' and 'ARRESTS_D'
# for 'ARRESTS' and 'ARRESTS_D', 0 indicates none so im setting -9 to none as that seems most likely to be the case if the information is missing
df1['ARRESTS'] = df1['ARRESTS'].replace(-9, 0)
df1['ARRESTS_D'] = df1['ARRESTS_D'].replace(-9, 0)


In [79]:
# time to clean up the data with ' STFIPS'
# for ' STFIPS', there are no null values in this column but we may want to remove it because the large values could through the accuracy off
df1['STFIPS'].notna().value_counts()

STFIPS
True    1722503
Name: count, dtype: int64

In [80]:
# time to clean up the data with 'REGION'
# for 'REGION', there are no null values in this column but we may want to remove it because the values could through the accuracy off and it doesnt seem to be relevant data
df1['REGION'].notna().value_counts()

REGION
True    1722503
Name: count, dtype: int64

In [81]:
# time to clean up the data with 'DIVISION'
# for 'DIVISION', there are no null values in this column but we may want to remove it because the values could through the accuracy off and it doesnt seem to be relevant data
df1['DIVISION'].notna().value_counts()

DIVISION
True    1722503
Name: count, dtype: int64

In [83]:
# time to clean up the data with 'SERVICES and SERVICES_D'
# for 'SERVICES and SERVICES_D', there are no null values in this column but we may want to remove it because the values could through the accuracy off and it doesnt seem to be relevant data for the questions we are trying to answer
df1['SERVICES'].notna().value_counts()
df1['SERVICES_D'].notna().value_counts()

SERVICES_D
True    1722503
Name: count, dtype: int64

In [84]:
# time to clean up the data with 'DAYWAIT'
# for 'DAYWAIT', its seems safe to replace a value of missing data with a value of '0' to indicate that they didnt wait
df1['DAYWAIT'] = df1['DAYWAIT'].replace(-9, 0)

In [85]:
# time to clean up the data with 'METHUSE'
# for 'METHUSE', its seems safe to replace a value of missing data with a value of 'none'
df1['METHUSE'] = df1['METHUSE'].replace(-9, 2)

In [86]:
# time to clean up the data with 'LOS'
# for 'LOS', stands for length of stay this data seems irrelivent to the questions we are trying to answer so I suggest dropping the column to perserve prediction accuracy
df1['LOS'].notna().value_counts()

LOS
True    1722503
Name: count, dtype: int64

In [ ]:
# time to clean up the data with PSOURCE: Referral source
# for 'PSOURCE' this doesnt seem to me to be relevant data so im replacing -9 with 1 for now to indicate that it was self motivated
df1['PSOURCE'] = df1['PSOURCE'].replace(-9, 1)

In [87]:
# time to clean up the data with DETCRIM: Detailed criminal justice referral
# for 'DETCRIM' im replacing -9 with 0 for now. most of this data is missing. we may want to drop this column
df1['DETCRIM'] = df1['DETCRIM'].replace(-9, 0)

In [89]:
# time to clean up the data with NOPRIOR: Previous substance use treatment episodes
# for 'NOPRIOR' im replacing -9 with 0 for now. its could go either way. we will ask clients
df1['NOPRIOR'] = df1['NOPRIOR'].replace(-9, 0)

In [93]:
# time to clean up the data with ROUTE1: Route of administration (primary)
# time to clean up the data with ROUTE2: Route of administration (secondary)
# time to clean up the data with ROUTE3: Route of administration (tertiary)
# for 'ROUTE1' im replacing -9 with 5 for now to indicate 'other'. its could go either way. we will ask clients
df1['ROUTE1'] = df1['ROUTE1'].replace(-9, 5)
df1['ROUTE2'] = df1['ROUTE2'].replace(-9, 5)
df1['ROUTE3'] = df1['ROUTE3'].replace(-9, 5)

In [92]:
# time to clean up the data with FRSTUSE1: Age at first use (primary)
# FRSTUSE1 is a column where its hard to determine what a value of -9 should be replaced with so im replacing it with 0 for now to have less of an impact on the data
df1['FRSTUSE1'] = df1['FRSTUSE1'].replace(-9, 0)

In [4]:
reason = df1['REASON']
df1_corr = df1.corr()
df1_corr.unstack().sort_values()
variable = df1_corr['REASON'].sort_values()
variable.tail(20)

MARSTAT                0.138023
FRSTUSE1               0.139035
VET                    0.144566
SERVICES_D             0.152773
SERVICES               0.154441
IDU                    0.155722
ROUTE1                 0.165564
EMPLOY                 0.177870
FREQ_ATND_SELF_HELP    0.183493
SUB1                   0.191675
EDUC                   0.193754
LIVARAG_D              0.206890
EMPLOY_D               0.208258
LIVARAG                0.209302
ARRESTS_D              0.209523
ARRESTS                0.209631
FREQ1_D                0.214312
SUB1_D                 0.243122
REASON                 1.000000
DISYR                       NaN
Name: REASON, dtype: float64

In [5]:
# new dataframe with only the top correlated farctors left
trimmed = df1[['SUB1_D', 'FREQ1_D', 'ARRESTS', 'ARRESTS_D', 'LIVARAG', 'EMPLOY_D', 'LIVARAG_D', 'EDUC', 'SUB1', 'FREQ_ATND_SELF_HELP', 'EMPLOY', 'ROUTE1', 'IDU', 'SERVICES', 'SERVICES_D', 'VET', 'FRSTUSE1', 'MARSTAT', 'REASON']]

In [6]:
y = trimmed["REASON"]
X = trimmed.copy()
X = X.drop(columns="REASON")
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
data = [X_train_scaled, X_test_scaled, y_train, y_test]

In [7]:
def test_model(model, data):
    X_train_scaled, X_test_scaled, y_train, y_test = data
    reg = model.fit(X_train_scaled, y_train)
    print(f'Model: {type(reg).__name__}')
    print(f'Train score: {reg.score(X_train_scaled, y_train)}')
    print(f'Test Score: {reg.score(X_test_scaled, y_test)}\n')
    plt.show()    

In [ ]:
test_model(LinearRegression(), data)
test_model(KNeighborsRegressor(), data)
test_model(RandomForestRegressor(), data)
test_model(ExtraTreesRegressor(), data)
test_model(AdaBoostRegressor(), data)
test_model(SVR(C=1.0, epsilon=0.2), data)

In [8]:
variable.head(20)

HLTHINS    -0.106469
PRIMINC    -0.098816
LOS        -0.084553
ALCFLG     -0.078218
PRIMPAY    -0.059634
AGE        -0.051736
DETNLF     -0.051458
CASEID     -0.042650
DIVISION   -0.030955
DAYWAIT    -0.014194
DETCRIM    -0.006041
BARBFLG    -0.003555
DSMCRIT    -0.002691
METHUSE    -0.001831
TRNQFLG     0.002683
INHFLG      0.003850
OTCFLG      0.005075
PCPFLG      0.006215
HALLFLG     0.007157
REGION      0.009529
Name: REASON, dtype: float64

In [9]:
y = df1["REASON"]
X = df1.copy()
X = X.drop(columns="REASON")
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
data = [X_train_scaled, X_test_scaled, y_train, y_test]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_scaled, y_train)
train = (f"Training Data Score: {logistic_regression_model.score(X_train_scaled, y_train)}")
test = (f"Testing Data Score: {logistic_regression_model.score(X_test_scaled, y_test)}")
testing_predictions = logistic_regression_model.predict(X_test)
print(accuracy_score(y_test, testing_predictions))

In [46]:
df2 = df.copy()
# Replace -9 with NaN
df2['FREQ1_D'] = df2['FREQ1_D'].replace(-9, 1)
df2['FREQ2_D'] = df2['FREQ2_D'].replace(-9, 1)
df2['FREQ3_D'] = df2['FREQ3_D'].replace(-9, 1)
# print('Replacing -9 with NaN. A -9 represents a missing value code in the dataset.')
# df2 = df2.replace(-9, np.nan)
# print('Replacing -9 with NaN. A -9 represents a missing value code in the dataset.')

#df2['FREQ1_D'].notna().value_counts()
# df2['FREQ1_D'].isna()

# for value in df2['FREQ1_D']:
#     if value == "nan":
#         value = 1
#         print(value)
#     else:
#          value = value
#          print(value)
# for value in df2['FREQ2_D']:
#     if value == 'NaN':
#         value = 1
#     else:
#         value = value
# for value in df2['FREQ3_D']:
#     if value == 'NaN':
#         value = 1
#     else:
#         value = value
        



In [47]:
df2['FREQ3_D'].head(20)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    3
Name: FREQ3_D, dtype: int64

In [22]:
df2.dropna()
df2

,DISYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,LOS,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2019,20191553576,2,NaN,4.0,1.0,7,NaN,37,1.0,...,0,0,0,0,0,0,9,4,0.0,1
1,2019,20191465214,2,NaN,3.0,1.0,7,NaN,35,1.0,...,0,0,0,0,0,0,9,4,0.0,3
2,2019,20191443889,2,NaN,2.0,1.0,7,NaN,35,1.0,...,0,0,0,0,0,0,9,4,0.0,3
3,2019,20191409377,2,NaN,3.0,1.0,7,NaN,37,1.0,...,0,0,0,0,0,0,9,4,0.0,3
4,2019,20191479567,2,NaN,3.0,3.0,7,NaN,37,1.0,...,0,0,0,0,0,0,9,4,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722498,2019,20191743528,56,NaN,4.0,2.0,7,NaN,33,1.0,...,0,0,0,0,0,0,8,4,0.0,1
1722499,2019,20191666713,56,NaN,3.0,3.0,7,1.0,13,7.0,...,0,0,0,0,0,0,8,4,0.0,2
1722500,2019,20191405666,56,NaN,2.0,1.0,6,NaN,33,1.0,...,0,0,0,0,0,0,8,4,0.0,2
1722501,2019,20191697509,56,NaN,1.0,1.0,7,3.0,14,7.0,...,0,0,0,0,0,0,8,4,0.0,3
